# Классификация с помощью генеративной модели

В этом ноутбуке я буду использовать модель T5 для классификации категории и тональности отзывов. В качестве промта буду подавать категорию и ее тональность через двоеточие.

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
from datasets import load_dataset
raw_datasets = load_dataset("alexcadillon/SemEval2014Task4", 'restaurants')
raw_datasets

Found cached dataset sem_eval2014_task4 (C:/Users/Kate/.cache/huggingface/datasets/alexcadillon___sem_eval2014_task4/restaurants/1.1.0/2486dc797a422f3346ce164d1b9a34b67f6f6c7b3f0f1e7ba9b536867f999049)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    trial: Dataset({
        features: ['sentenceId', 'text', 'aspectTerms', 'aspectCategories'],
        num_rows: 100
    })
    train: Dataset({
        features: ['sentenceId', 'text', 'aspectTerms', 'aspectCategories'],
        num_rows: 3041
    })
    test: Dataset({
        features: ['sentenceId', 'text', 'aspectTerms', 'aspectCategories'],
        num_rows: 800
    })
})

In [3]:
def combo_label(example):   
    labels_combo = []
    for item in example['aspectCategories']:
        labels_combo.append(f"{item['category']}: {item['polarity']}")
    example['labels_combo'] = labels_combo
    return example

In [4]:
updated_datasets = raw_datasets.map(combo_label)

Loading cached processed dataset at C:\Users\Kate\.cache\huggingface\datasets\alexcadillon___sem_eval2014_task4\restaurants\1.1.0\2486dc797a422f3346ce164d1b9a34b67f6f6c7b3f0f1e7ba9b536867f999049\cache-3a86ab4434787b0e.arrow
Loading cached processed dataset at C:\Users\Kate\.cache\huggingface\datasets\alexcadillon___sem_eval2014_task4\restaurants\1.1.0\2486dc797a422f3346ce164d1b9a34b67f6f6c7b3f0f1e7ba9b536867f999049\cache-4827cdc485eb1a5d.arrow
Loading cached processed dataset at C:\Users\Kate\.cache\huggingface\datasets\alexcadillon___sem_eval2014_task4\restaurants\1.1.0\2486dc797a422f3346ce164d1b9a34b67f6f6c7b3f0f1e7ba9b536867f999049\cache-8c522bf7b72d8186.arrow


In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")

In [10]:
def preprocess_data(example):
  # take a batch of texts
  text = example["text"]
  # encode them
  encoding = tokenizer.encode_plus(text, padding = True)
  encoding["labels"] = tokenizer.encode_plus(', '.join(example['labels_combo']))
  encoding["labels"] = encoding["labels"]['input_ids']

  return encoding

In [11]:
preprocess_data(updated_datasets['train'][0])

{'input_ids': [299, 8, 871, 47, 78, 17425, 12, 178, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [313, 10, 2841, 1]}

In [12]:
tokenized_dataset = updated_datasets.map(preprocess_data, remove_columns=updated_datasets['train'].column_names)
tokenized_dataset = tokenized_dataset.remove_columns(['attention_mask'])
tokenized_dataset

Loading cached processed dataset at C:\Users\Kate\.cache\huggingface\datasets\alexcadillon___sem_eval2014_task4\restaurants\1.1.0\2486dc797a422f3346ce164d1b9a34b67f6f6c7b3f0f1e7ba9b536867f999049\cache-dc31ad5d29e23710.arrow
Loading cached processed dataset at C:\Users\Kate\.cache\huggingface\datasets\alexcadillon___sem_eval2014_task4\restaurants\1.1.0\2486dc797a422f3346ce164d1b9a34b67f6f6c7b3f0f1e7ba9b536867f999049\cache-1bda3d200e39ec3e.arrow
Loading cached processed dataset at C:\Users\Kate\.cache\huggingface\datasets\alexcadillon___sem_eval2014_task4\restaurants\1.1.0\2486dc797a422f3346ce164d1b9a34b67f6f6c7b3f0f1e7ba9b536867f999049\cache-476da8de032ba463.arrow


DatasetDict({
    trial: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 100
    })
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 3041
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 800
    })
})

In [13]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    evaluation_strategy = 'epoch',  
    learning_rate = 2e-3,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    weight_decay = 0.01,
    num_train_epochs = 8,
    predict_with_generate = True,
    push_to_hub = False,  
    output_dir = "outputs"   
    
)

In [14]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)

In [15]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(model,
                        args,
                        train_dataset = tokenized_dataset['train'],
                        eval_dataset = tokenized_dataset['test'],
                        data_collator = data_collator,
                        tokenizer = tokenizer,)

trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.257915
2,No log,0.220764
3,0.241900,0.218035
4,0.241900,0.229225
5,0.241900,0.242655
6,0.100200,0.273756
7,0.100200,0.302112
8,0.035000,0.325696


TrainOutput(global_step=1528, training_loss=0.12381565637145367, metrics={'train_runtime': 286.1417, 'train_samples_per_second': 85.021, 'train_steps_per_second': 5.34, 'total_flos': 1374876408462336.0, 'train_loss': 0.12381565637145367, 'epoch': 8.0})

In [16]:
test = trainer.predict(tokenized_dataset['test'])

In [17]:
updated_datasets['test']

Dataset({
    features: ['sentenceId', 'text', 'aspectTerms', 'aspectCategories', 'labels_combo'],
    num_rows: 800
})

In [18]:
count = 0
total_texts = []
for prediction, label in zip(test.predictions, updated_datasets['test']['labels_combo']):
    if tokenizer.decode(prediction, skip_special_tokens = True) == ', '.join(label):
        count += 1
    total_texts.append((tokenizer.decode(prediction, skip_special_tokens = True),', '.join(label)))
print(f'Total correct answers {count/len(total_texts)}')
print('==============')
for i in total_texts:
    if i[0] != i[1]:
        print(i)
        print('==============')

Total correct answers 0.645
('food: negative', 'food: positive')
('food: positive, ambience: positive', 'food: conflict, ambience: positive')
('food: positive, service: conflict', 'food: positive, service: negative')
('food: neutral', 'food: negative')
('food: neutral', 'food: negative')
('food: negative', 'food: neutral')
('service: positive', 'service: positive, anecdotes/miscellaneous: positive')
('anecdotes/miscellaneous: neutral', 'anecdotes/miscellaneous: positive')
('anecdotes/miscellaneous: positive', 'anecdotes/miscellaneous: neutral')
('anecdotes/miscellaneous: positive, food: positive', 'anecdotes/miscellaneous: positive')
('anecdotes/miscellaneous: positive', 'food: positive')
('food: positive, service: positive', 'ambience: positive, service: positive, food: positive')
('ambience: negative', 'food: negative')
('food: positive, service: positive', 'service: positive, food: positive')
('food: positive', 'food: neutral')
('food: positive, ambience: positive, price: positive',